## 全データへの言語＋日付正規化

In [1]:
import pandas as pd

file_path = "fact_check_insights/claim_review.csv"
df = pd.read_csv(file_path)#244560

C:\Users\momo1\AppData\Local\Temp\ipykernel_112248\802964034.py:4: DtypeWarning: Columns (14,24,25,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)#244560


In [ ]:
import pandas as pd
from langdetect import detect, DetectorFactory

# 安定した言語検出のためにランダムシードを設定
DetectorFactory.seed = 0
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

df['language'] = df['claimReviewed'].apply(detect_language)

In [55]:
df["datePublished"] = pd.to_datetime(df["datePublished"], errors='coerce',utc=True)
df["itemReviewed.datePublished"] = pd.to_datetime(df["itemReviewed.datePublished"], errors='coerce',utc=True)

In [77]:
# from mymodule import read_bin,save_bin
# save_bin("new_data2/all_df_lang.bin",df)

## 団体（英）ごとのデータ件数可視化

In [114]:
from mymodule import read_bin,save_bin
# df = read_bin("new_data_2/all_df_lang.bin")

In [115]:
df = df[df['language'] == "en"]

In [116]:
# 使用する列のみ選択
df = df[['id', 'datePublished', 'url',
         'itemReviewed.datePublished','claimReviewed',
         'author.name',
         'reviewRating.ratingValue',
         'reviewRating.bestRating',
         'reviewRating.worstRating',
         'reviewRating.ratingExplanation',
         "reviewRating.alternateName"]]

In [117]:
# l = ["PolitiFact","FACTLY","Check Your Fact","VERIFY","Lead Stories LLC","AFP"]
l = ["PolitiFact","FACTLY","FactCheck.org","Check Your Fact","VERIFY","Lead Stories LLC"]
df = df[df['author.name'].isin(l)]

In [118]:
# 列をintに変換
import pandas as pd
def convert_columns_to_int(df, col):
    def convert_to_int(x):
        try:
            return int(float(x))
        except ValueError:
            return float('nan')
            
    df[col] = df[col].apply(convert_to_int).astype('Int64')
    return df

In [119]:
df = convert_columns_to_int(df,"reviewRating.ratingValue")
df = df.fillna(-999)

# save_bin("new_data_2/df_AB.bin",df)

In [25]:
# import pandas as pd
# specific_date = '2023-11-01'  ## GPT-4o（gpt-4o-2024-08-06）以降:13084
# df_A = df[df['itemReviewed.datePublished'] >= pd.to_datetime(specific_date,utc=True)]#after:3702
# df_B = df[df['datePublished'] < pd.to_datetime(specific_date,utc=True)]#before:25449

In [62]:
# df_A["author.name"].value_counts().reset_index()

In [63]:
# df_B["author.name"].value_counts().reset_index()

In [10]:
# save_bin("new_data_2/df_A.bin",df_A)
# save_bin("new_data_2/df_B.bin",df_B)

In [28]:
# del df_A,df_B

## データ整形

In [171]:
# df = read_bin("new_data_2/df_AB.bin")

In [167]:
# index_counter(df,"reviewRating.ratingValue")

In [126]:
dfs = {category.replace(" ","_"): group for category, group in df.groupby('author.name')}

In [181]:
def show_df(the_df,col_name,query):
    the_df_1 = the_df[the_df[col_name] == query]
    return the_df_1
    
def count_by_categories(df, cols):
    if isinstance(cols, list):
        return df.groupby(cols).size().reset_index(name='count')
    if isinstance(cols, str):
        return df[cols].value_counts().reset_index()

### PolitiFact

In [148]:
# temp_df = dfs["PolitiFact"]

In [153]:
temp_df.loc[(temp_df['reviewRating.ratingValue'] == -999)
            &(temp_df['reviewRating.alternateName'] == "True"),
            'reviewRating.ratingValue'] = 0
temp_df.loc[(temp_df['reviewRating.ratingValue'] == -999)
            &(temp_df['reviewRating.alternateName'] == "Mostly True"),
            'reviewRating.ratingValue'] = 1
temp_df.loc[(temp_df['reviewRating.ratingValue'] == -999)
            &(temp_df['reviewRating.alternateName'] == "Half True"),
            'reviewRating.ratingValue'] = 2
temp_df.loc[(temp_df['reviewRating.ratingValue'] == -999)
            &(temp_df['reviewRating.alternateName'] == "Mostly False"),
            'reviewRating.ratingValue'] = 3
temp_df.loc[(temp_df['reviewRating.ratingValue'] == -999)
            &(temp_df['reviewRating.alternateName'] == "False"),
            'reviewRating.ratingValue'] = 4
temp_df.loc[(temp_df['reviewRating.ratingValue'] == -999)
            &(temp_df['reviewRating.alternateName'] == "Pants on Fire"),
            'reviewRating.ratingValue'] = 5

# 6種類以外の自由記述ラベルを表示したいとき用のプログラム
# temp_df_2 = temp_df[temp_df["reviewRating.ratingValue"]==-999]
# list(count_by_categories(temp_df_2,["reviewRating.alternateName"])["reviewRating.alternateName"])

temp_df = temp_df[~temp_df["reviewRating.ratingValue"].isin([8,7,6,-999])]
#政治家の一貫性レベルを表すラベルを削除：No Flip/Half Flip/Full Flop

temp_df['reviewRating.bestRating'] = 0
temp_df['reviewRating.worstRating'] = 5

In [176]:
count_by_categories(temp_df,["reviewRating.ratingValue","reviewRating.alternateName"])

,reviewRating.ratingValue,reviewRating.alternateName,count
0,0,True,217
1,1,Mostly True,366
2,2,Half True,494
3,3,Mostly False,716
4,4,False,3630
5,5,Pants on Fire,1243


In [177]:
# dfs["PolitiFact"] = temp_df

### FACTLY

In [193]:
# temp_df = dfs["FACTLY"]

In [212]:
# 表記の空白等 可視化用
# list(count_by_categories(temp_df,["reviewRating.ratingValue","reviewRating.alternateName"])["reviewRating.alternateName"])
temp_df.loc[(temp_df['reviewRating.ratingValue'] == 1)
            &(temp_df['reviewRating.alternateName'].isin([" FALSE",'FALSE','FALSE ','FALSSE',"FALSR"])),
            'reviewRating.alternateName'] = "FALSE"
temp_df.loc[(temp_df['reviewRating.ratingValue'] == 2)
            &(temp_df['reviewRating.alternateName'].isin(["ISLEADING",'MIISLEADING','MILSEADING','MISLEADING',"MISLEADING ","MISLEADING   "])),
            'reviewRating.alternateName'] = "MISLEADING"
temp_df.loc[(temp_df['reviewRating.ratingValue'] == 3)
            &(temp_df['reviewRating.alternateName'].isin(["UNVERIFIED",'UNVERIFIED '])),
            'reviewRating.alternateName'] = "UNVERIFIED"

target_combinations = [(1,"FALSE"), (2,"MISLEADING"),(3,"UNVERIFIED"),(4,"PARTLY TRUE"),(5,"TRUE")]
temp_df = temp_df[temp_df.apply(lambda x: (x['reviewRating.ratingValue'], x['reviewRating.alternateName']) in target_combinations, axis=1)]

temp_df['reviewRating.bestRating'] = 5
temp_df['reviewRating.worstRating'] = 1

In [213]:
count_by_categories(temp_df,["reviewRating.ratingValue","reviewRating.alternateName"])

,reviewRating.ratingValue,reviewRating.alternateName,count
0,1,FALSE,3598
1,2,MISLEADING,1052
2,3,UNVERIFIED,22
3,4,PARTLY TRUE,14
4,5,TRUE,62


In [215]:
# dfs["FACTLY"] = temp_df

### FactCheck.org

In [233]:
# 不適格により削除
# del dfs["FactCheck.org"]

### Check Your Fact

In [235]:
# temp_df = dfs["Check_Your_Fact"]

In [237]:
temp_df['reviewRating.bestRating'] = 3
temp_df['reviewRating.worstRating'] = 1

In [236]:
count_by_categories(temp_df,["reviewRating.ratingValue","reviewRating.alternateName"])

,reviewRating.ratingValue,reviewRating.alternateName,count
0,1,False,2391
1,2,Unsubstantiated,41
2,3,True,28


In [241]:
# dfs["Check_Your_Fact"]=temp_df

### VERIFY

In [243]:
# temp_df = dfs["VERIFY"]

In [251]:
temp_df['reviewRating.bestRating'] = 5
temp_df['reviewRating.worstRating'] = 1

In [245]:
count_by_categories(temp_df,["reviewRating.ratingValue"])

,reviewRating.ratingValue,count
0,1,2442
1,2,63
2,3,31
3,4,274
4,5,1961


In [253]:
# dfs["VERIFY"] = temp_df

### Lead Stories LLC

In [254]:
# 特段加工は不要と判断，そのまま
# temp_df = dfs["Lead_Stories_LLC"]

In [256]:
count_by_categories(temp_df,["reviewRating.ratingValue"])

,reviewRating.ratingValue,count
0,1,9371
1,2,19
2,3,382
3,4,8
4,5,66


In [265]:
# show_df(temp_df,"reviewRating.ratingValue",3)

### AFP

In [266]:
# ラベルへの不信感，使用中断
temp_df = dfs["AFP"]

In [269]:
count_by_categories(temp_df,["reviewRating.ratingValue"])

,reviewRating.ratingValue,count
0,1,3303
1,2,819
2,3,358
3,4,1
4,5,34


In [277]:
# del dfs["AFP"]

In [278]:
save_bin("new_data_2/dfs_AB.bin",dfs)

In [ ]:
# もっとシンプルな予備実験でよいのでは？